In [2]:
from CoolProp.CoolProp import PropsSI as COOLPROP
import math
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d.axes3d import get_test_data

#Listas
registro_fluidos = []
COP_lista = []
lista_T5 = []
Wcomp_total_lista = []
cont = []

#Lista de Temperaturas
T5 = 256.15

#Lista de características para as curvas do gráfico
lista_cor = ['r','g','b', 'c', 'm']
lista_ls = ['-', '-.', ':']
caracteristicas = []
for ls in lista_ls:
    for cor in lista_cor:
        caracteristicas.append((cor, ls))   
    
#Parâmetros
Pfrig = 120 * 1000
T1 = -70 + 273.15
T13 = 25 + 273.15
N = 0.7 #Eficiência isentrópica
fluid_high_lista = ['R134a', 'R717', 'R290', 'R600']
fluid_low_lista = ['R23', 'R170', 'NitrousOxide']
drop_suc = 0 #[Pa]
drop_desc = 0 #[Pa]
DeltaT = 0.01 #[°C]

#Loop nos dois ciclos - variação dos refrigerantes e do coeficiente politrópico
for fluid_low in fluid_low_lista:
    for fluid_high in fluid_high_lista:
        par_fluido = (fluid_low, fluid_high)
        registro_fluidos.append(par_fluido)
         #Temperatura T5
        T5 = 256.15
        #Pressão crítica do fluido superior
        P_crit_high = COOLPROP('PCRIT', fluid_high)
        #Ciclo inferior
        P1 = COOLPROP('P', 'T', T1, 'Q', 1, fluid_low)
        P5 = COOLPROP('P', 'T', T5, 'Q', 0, fluid_low)
        h5 = COOLPROP('H', 'T', T5, 'Q', 0, fluid_low)
        h6 = h5
        #Pressão intermediária
        P_int =  (P1 * P5 * T5 / T1) ** 0.5
        P6 = P_int
        P3 = P_int
        P7 = P_int
        P2 = P_int
        h7 = COOLPROP('H', 'P', P7, 'Q', 0, fluid_low)
        h8 = h7
        h3 = COOLPROP('H', 'P', P3, 'Q', 1, fluid_low)
        h1 = COOLPROP('H', 'T', T1, 'Q', 1, fluid_low)
        #CBP
        ha = h1
        Pa = P1 - drop_suc
        Ta = COOLPROP('T', 'H', ha, 'P', Pa, fluid_low)
        Pb = Pa
        Tb = Ta + DeltaT
        hb = COOLPROP('H', 'T', Tb, 'P', Pb, fluid_low)
        Sb = COOLPROP('S', 'T', Tb, 'P', Pb, fluid_low)
        Pc = P2 + drop_desc
        Sc_ideal = Sb     
        hc_ideal = COOLPROP('H', 'S', Sc_ideal , 'P', Pc, fluid_low)
        hc_real = (hc_ideal - hb)/N + hb
        h2 = hc_real
            
        m_CBP_inf = Pfrig / (h1 - h8)
        #CAP
        hw = h3
        Pw = P3 - drop_suc
        Tw = COOLPROP('T', 'H', hw, 'P', Pw, fluid_low)
        Px = Pw
        Tx = Tw + DeltaT
        hx = COOLPROP('H', 'P', Px, 'T', Tx, fluid_low)
        sx = COOLPROP('S', 'P', Px, 'T', Tx, fluid_low)
        P4 = P5
        Py = P4 + drop_desc
        sy_ideal = sx
        hy_ideal = COOLPROP('H', 'P', Py, 'S', sy_ideal, fluid_low)
        hy_real = (hy_ideal - hx)/N + hx
        h4 = hy_real
            
        #Ciclo superior
        T16 = T5 - 5
        P16 = COOLPROP('P', 'T', T16, 'Q', 0, fluid_high)
        P9 = P16
        T9 = COOLPROP('T', 'P', P9, 'Q', 1, fluid_high)
            
        P13 = COOLPROP('P', 'T', T13, 'Q', 0, fluid_high)
        h13 = COOLPROP('H', 'T', T13, 'Q', 0, fluid_high)
        h14 = h13
        P12 = P13
        #Pressão intermediária Superior
        P_int_sup = (P9 * P13 * T13 / T9) ** 0.5
        P10 = P_int_sup
        P11 = P_int_sup
        P14 = P_int_sup
        P15 = P_int_sup
        h15 = COOLPROP('H', 'P', P15, 'Q', 0, fluid_high)
        h16 = h15
        h11 = COOLPROP('H', 'P', P7, 'Q', 1, fluid_high)
        h9 = COOLPROP('H', 'T', T9, 'Q', 1, fluid_high)
        #CBP_sup
        hm = h9
        Pm = P9 - drop_suc
        Tm = COOLPROP('T', 'H', hm, 'P', Pm, fluid_high)
        Pn = Pm
        Tn = Tm + DeltaT

        hn = COOLPROP('H', 'P', Pn, 'T', Tn, fluid_high)
        sn = COOLPROP('S', 'P', Pn, 'T', Tn, fluid_high)
        Po = P10 + drop_desc
        so_ideal = sn
        ho_ideal = COOLPROP('H', 'P', Po, 'S', so_ideal, fluid_high)
        ho_real = (ho_ideal - hn)/N + hn
        h10 = ho_real
            
        #CAP_sup
        he = h11
        Pe = P11 - drop_suc
        Te = COOLPROP ('T', 'H', he, 'P', Pe, fluid_high)
        Pf = Pe
        Tf = Te + DeltaT
            
        hf = COOLPROP('H', 'P', Pf, 'T', Tf, fluid_high)
        sf = COOLPROP('S', 'P', Pf, 'T', Tf, fluid_high)
        Pg = P12 + drop_desc
        sg_ideal = sf
        hg_ideal = COOLPROP('H', 'P', Pg, 'S', sg_ideal, fluid_high)
        hg_real = (hg_ideal - hf)/N + hf
        h12 = hg_real
       
        #Ciclo de baixa pressão
        #Trabalho específico do CBP_inf
        w_CBP_inf = hc_real - hb
        #Trabalho do CBP_inf
        W_CBP_inf = m_CBP_inf * w_CBP_inf
        #CAP_inf
        m_CAP_inf = m_CBP_inf * (h7 - h2) / (h6 - h3)
        #Trabalho específico do CAP
        w_CAP_inf = hy_real - hx
        #Trabalho do CAP_inf
        W_CAP_inf = m_CAP_inf * w_CAP_inf
        #Trabalho total no ciclo de baixa pressão
        W_low_total = W_CAP_inf + W_CBP_inf
        #Calor trocado no trocador central
        Qcond_low =  m_CAP_inf * (h4 - h5)
         
        #Ciclo de alta pressão
        Qevap_high = Qcond_low
        #CBP_sup
        m_CBP_sup = Qevap_high / (h9 - h16)
        #Trabalho específico do CBP_sup
        w_CBP_sup = ho_real - hn
        #Trabalho do CBP_sup
        W_CBP_sup = w_CBP_sup * m_CBP_sup
        #CAP_sup
        m_CAP_sup = m_CBP_sup * (h15 - h10) / (h14 - h11)
        if P_crit_high > P12:
            lista_T5.append(T5)
            #Trabalho específico do CAP_sup
            w_CAP_sup = hg_real - hf
            #Trabalho do CAP_sup
            W_CAP_sup = m_CAP_sup * w_CAP_sup
            #Trabalho total do ciclo de alta pressão
            W_high_total = W_CAP_sup + W_CBP_sup
            #Trabalho total do ciclo
            Wcomp_total = W_low_total + W_high_total
            Wcomp_total_lista.append(Wcomp_total / 1000)
            #COP
            COP = Pfrig / Wcomp_total
            COP_lista.append(COP)
            print(COP)
            print(P_int)
            print("A", fluid_high)
            print("B", fluid_low)
            print("P4",P4)
        else:
            print ('A pressão P12 = {:0.2f} kPa (T5 = {}, fluido inf = {}, fluido sup = {}) é maior do que a ' 
            'pressão crítica do {} logo o ponto não foi incluído no gráfico'.format(P12 / 1000,T5,fluid_low,
                                                                                               fluid_high, fluid_high))       
 

1.1171986765788084
611457.2724593427
A R134a
B R23
P4 1530846.5342272525
1.1190765677315224
611457.2724593427
A R717
B R23
P4 1530846.5342272525
1.111698460077873
611457.2724593427
A R290
B R23
P4 1530846.5342272525
1.1403569548243189
611457.2724593427
A R600
B R23
P4 1530846.5342272525
1.1226539826111595
697517.792070804
A R134a
B R170
P4 1543865.1621415792
1.1234356544736106
697517.792070804
A R717
B R170
P4 1543865.1621415792
1.1170572756043824
697517.792070804
A R290
B R170
P4 1543865.1621415792
1.14700955352712
697517.792070804
A R600
B R170
P4 1543865.1621415792
1.118217715264301
821426.1167978488
A R134a
B NitrousOxide
P4 1963060.1245940677
1.117663931048516
821426.1167978488
A R717
B NitrousOxide
P4 1963060.1245940677
1.1126214369623177
821426.1167978488
A R290
B NitrousOxide
P4 1963060.1245940677
1.143689890962856
821426.1167978488
A R600
B NitrousOxide
P4 1963060.1245940677
